In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import re

In [37]:
def img_2_array(file):
    y = re.findall('/(\w)_\d+\.\w+', file)[0]
    img = Image.open(file)
    array = list(np.array(img).flatten())
    return y, array

In [3]:
image_path = '../data/converted_images'
files = os.listdir(image_path)
X = list()
for file in files:
    path = os.path.join(image_path, file)
    y = re.split('_(\d+)', path)[0][-1]
    img = Image.open(path)
    array = list(np.array(img).flatten())
    array.append(y)
    X.append(array)
df = pd.DataFrame(X)
df.columns = ['pixel{}'.format(i) for i in range(1, 433)] + ['label']

In [4]:
df.to_csv('../data/data_sets.csv', index=None)

In [2]:
df = pd.read_csv('../data/data_sets.csv')
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
# df.head()

In [3]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
le = LabelEncoder()
y_le = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_le, test_size=0.3, random_state=1)

/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
from sklearn.grid_search import GridSearchCV
pipe_svm = Pipeline([('mm', MinMaxScaler()),
                   ('clf', SVC(random_state=1))])
param_range = [0.01, 0.1, 1.0, 10.0, 100.0]
param_grid = [{'clf__C': param_range,
             'clf__gamma': param_range,
             'clf__kernel': ['rbf']}]
gs = GridSearchCV(estimator=pipe_svm, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/

/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/

KeyboardInterrupt: 

In [5]:
pipe_rf = Pipeline([('mm', MinMaxScaler()),
                   ('clf', RandomForestClassifier(random_state=1, n_estimators=1000, n_jobs=-1, criterion='entropy'))])
pipe_rf.fit(X_train, y_train)
pipe_rf.score(X_test, y_test)

/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


0.9926508732491787

In [31]:
y_pred = pipe_rf.predict(X_test)
le.inverse_transform(y_pred)

/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['N', '7', '7', ..., 'M', 'E', '2'], dtype=object)

In [42]:
y, X = img_2_array('../captcha_handle/3_3.jpg')
y_pred = pipe_rf.predict([X])
print(y_pred)
y_true = le.inverse_transform(y_pred)
print(y_true)

[1]
['3']


/home/liuchao/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [6]:
import pickle
pickle.dump(pipe_rf, open('random_forest_model', 'wb'), protocol=4)

In [7]:
import pickle
pickle.dump(le, open('label_encoder', 'wb'), protocol=4)

In [7]:
import pickle
from sklearn.metrics import classification_report
model = pickle.load(open('random_forest_model', 'rb'))
le = pickle.load(open('label_encoder', 'rb'))
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

             precision    recall  f1-score   support

          0       0.99      1.00      1.00       371
          1       0.99      0.99      0.99       374
          2       1.00      1.00      1.00       339
          3       1.00      0.99      0.99       350
          4       0.99      0.99      0.99       345
          5       1.00      0.99      1.00       399
          6       1.00      0.99      0.99       364
          7       1.00      0.99      0.99       395
          8       1.00      1.00      1.00       366
          9       0.99      1.00      1.00       354
         10       0.99      1.00      0.99       365
         11       0.99      0.99      0.99       330
         12       1.00      0.99      1.00       373
         13       1.00      0.99      0.99       358
         14       1.00      0.99      0.99       350
         15       1.00      0.99      1.00       355
         16       0.94      1.00      0.97       326
         17       0.99      0.99      0.99   